In [1]:
import os
import numpy as np
import multiprocessing as mp
from mg5_tools import semilla, run_mg5_file

UFO_name = "Mod2_VLQ_UFO"
case = "woRHC_UpZp"
output_folder= os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "SIMULATIONS", f"lq_zp_{case}")
param_cards_folder = os.path.join(os.getcwd(), f"Paramcards_{case}")
os.makedirs(output_folder, exist_ok=True)

n_events = 0.8e6

nevts = 50e3
n_runs = int(n_events/nevts)
n_cores = mp.cpu_count()-2

mass_step = 500 # GeV

# Leptoquark parameters
M_U = np.arange(1000, 3500 + mass_step, mass_step)
g_U = np.array([1.5])

In [2]:
headers = {
"zp_tau_tau" : 
f"""
import model {UFO_name} 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > zp, zp > ta ta @0 QED = 0 QCD = 0 
""",
"ta_ta" :
f"""
import model {UFO_name}
define ta = ta+ ta-
generate p p > ta ta @0 QED = 0 QCD = 0 $ zp
""",
"lq_lq" :
f"""
import model {UFO_name}
define lq = vlq vlq~
generate p p > lq lq @0
""",
"lq_ta" :
f"""
import model {UFO_name}
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > lq ta @0
""",
}

kin_gen_cuts = {
    "cut_decays" : True,
    "ptb" : 30,
    "ptj" : 20,
    "ptl" : 20,
    "etab" : 2.5,
    "pt_min_pdg" : "{15: 30}",
    "eta_max_pdg" : '{15: 2.5}',
    "mmll" : 20
}

In [ ]:
from itertools import product

log_file = open (os.path.join(os.getcwd(), "generation.log"), 'w')

for m, g in product(M_U, g_U):
    seeds=[]
    for header in headers:
        if "lq_lq" == header: continue
        if "lq_ta" == header: continue
        label = f'M{m}_gU{g:.4f}'.replace('.', '_')
        mg5_outpuut_folder = os.path.join(output_folder, header,label)
        os.makedirs(mg5_outpuut_folder, exist_ok=True)
        
        proc_file_path = os.path.join(mg5_outpuut_folder, f"proc_script.mg5")
        with open(proc_file_path, 'w') as f:
            f.write(headers[header])
            f.write(f"output {mg5_outpuut_folder} -nojpeg\n")
        run_mg5_file(proc_file_path)

        param_card = os.path.join(param_cards_folder, f"param_card_{label}.dat")
        delphes_card = os.path.join(os.getcwd(), "delphes_card_CMS-0.7bTag.dat")
        pythia_card = os.path.join(os.getcwd(), "pythia8_card.dat")
        
        if not os.path.isfile(param_card):
            raise Exception(f"Param card {os.path.basename(param_card)} not found")
        
        for i in range(n_runs):
            
            f = open(os.path.join(mg5_outpuut_folder, f"run_{i}.mg5"), 'w')
            f.write(f"launch {mg5_outpuut_folder} -m\n")
            f.write(f"{n_cores}\n") 
            f.write("shower = Pythia8 \n")
            f.write("detector = Delphes \n")
            f.write("done \n")
            f.write(f"set nevents {nevts}\n")
            
            seed = semilla(seeds)
            f.write(f"set iseed {seed}\n")
            [f.write(f"set {key} {value}\n") for key, value in kin_gen_cuts.items()]
            f.write(f"{param_card}\n")
            f.write(f"{delphes_card}\n")
            f.write(f"{pythia_card}\n")
            f.write("done \n")
            f.close()

            run_mg5_file(os.path.join(mg5_outpuut_folder, f"run_{i}.mg5"))
    log_file.write(f"Done for M_U = {m}, g_U = {g}")
log_file.close()